In [1]:
from pytezos import ContractInterface, pytezos, MichelsonRuntimeError

In [2]:
contract_fn = '../../build/tz/baking_bet.tz'

a='tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ'
b='tz1MdaJfWzP5pPx3gwPxfdLZTHW6js9havos'
c='tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE'
d='tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM'

contract = ContractInterface.from_file(contract_fn)

In [3]:
lambda_fn = '../../build/tz/lambda_raise_liq_fee.tz'
lambda_code = open(lambda_fn).read()

In [4]:
import time

RUN_TIME = int(time.time())
ONE_HOUR = 60*60
ONE_DAY = 24*ONE_HOUR

expiration_fee = 100_000
measure_start_fee = 100_000
oracle_address = 'KT1RCNpUEDjZAYhabjzgz1ZfxQijCDVMEaTZ'


default_config = {
    'defaultTime': 0,
    'expirationFee': expiration_fee,
    'minLiquidityPercent': 0,
    'maxLiquidityPercent': 300_000,  # 30% for 1_000_000 liquidityPrecision
    'maxAllowedMeasureLag': ONE_HOUR*4,  # 4 hours
    'maxMeasurePeriod': ONE_DAY*31,  # 31 day
    'maxPeriodToBetsClose': ONE_DAY*31,  # 31 day
    'measureStartFee': measure_start_fee,
    'minMeasurePeriod': 60*5,  # 5 min
    'minPeriodToBetsClose': 60*5,  # 5 min
    'oracleAddress': oracle_address,
    'rewardCallFee': 100_000,
}

init_storage = {
    'events': {},
    'betsAboveEq': {},
    'betsBellow': {},
    'providedLiquidityAboveEq': {},
    'providedLiquidityBellow': {},
    'liquidityShares': {},
    'depositedBets': {},
    'lastEventId': 0,
    'closeCallId': None,
    'measurementStartCallId': None,
    'newEventConfig': default_config,
    'manager': a,

    'liquidityPrecision': 1_000_000,
    'ratioPrecision': 100_000_000,
    'sharePrecision': 100_000_000,
    'targetDynamicsPrecision': 1_000_000,

    'bakingRewards': 0
}

event = {
    'currencyPair': 'XTZ-USD',
    'targetDynamics': 1_000_000,
    'betsCloseTime': RUN_TIME + 24*ONE_HOUR,
    'measurePeriod': 12*ONE_HOUR,
    'liquidityPercent': 0,
}

In [5]:
result = contract.newEvent(event).with_amount(200_000).interpret(
    now=RUN_TIME, storage=init_storage)

In [6]:
result.storage['newEventConfig']['minLiquidityPercent']

0

In [7]:
result = contract.setDelegate('tz3e7LbZvUtoXhpUD1yb6wuFodZpfYRb9nWJ').interpret(
    now=RUN_TIME, storage=init_storage)

result.operations

[{'kind': 'delegation',
  'source': 'KT1BEqzn5Wx8uJrZNvuS9DVHmLvG9td3fDLi',
  'delegate': 'tz3e7LbZvUtoXhpUD1yb6wuFodZpfYRb9nWJ'}]

In [8]:
result = contract.updateConfig(lambda_code).interpret(
    now=RUN_TIME, storage=result.storage, sender=a)

In [15]:
# pytezos.transaction(amount=200_000)

In [11]:
contract#.transaction()


Properties
.block_id	head
.storage	# access storage data at block `block_id`
.parameter	# root entrypoint

Entrypoints
.bet()
.claimBakingRewards()
.close()
.closeCallback()
.default()
.newEvent()
.provideLiquidity()
.setDelegate()
.startMeasurement()
.startMeasurementCallback()
.triggerForceMajeure()
.updateConfig()
.withdraw()
.root()

Helpers
.big_map_get()
.create_from()
.from_context()
.from_file()
.from_micheline()
.from_michelson()
.from_url()
.metadata()
.metadata_url()
.operation_result()
.originate()
.program()
.script()
.to_file()
.to_micheline()
.to_michelson()
.using()

In [48]:
result.storage['newEventConfig']['liquidityPercent']

10000

In [6]:
provide_params = {
    'eventId': 0,
    'expectedRatioAgainst': 1,
    'expectedRatioFor': 1,
    'maxSlippage': 100_000,
}

result = contract.provideLiquidity(provide_params).with_amount(10_000_000).interpret(
    sender=d, now=RUN_TIME, storage=result.storage)
result.storage['events'][0]

{'betsAgainstLiquidityPoolSum': 5000000,
 'betsCloseTime': 1620373402,
 'betsForLiquidityPoolSum': 5000000,
 'closedDynamics': 0,
 'closedOracleTime': 1530342452,
 'closedRate': 0,
 'createdTime': 1620287002,
 'currencyPair': 'XTZ-USD',
 'expirationFee': 200000,
 'firstProviderAgainstSharesSum': 5000000,
 'firstProviderForSharesSum': 5000000,
 'isBetsForWin': False,
 'isClosed': False,
 'isMeasurementStarted': False,
 'liquidityPercent': 10000,
 'liquidityPrecision': 1000000,
 'measureOracleStartTime': 1530342452,
 'measurePeriod': 43200,
 'measureStartFee': 200000,
 'oracleAddress': 'KT1SUP27JhX24Kvr11oUdWswk7FnCW78ZyUn',
 'ratioPrecision': 100000000,
 'rewardCallFee': 100000,
 'sharePrecision': 100000000,
 'startRate': 0,
 'targetDynamics': 1000000,
 'targetDynamicsPrecision': 1000000,
 'totalLiquidityAgainstSharesSum': 5000000,
 'totalLiquidityForSharesSum': 5000000,
 'totalLiquidityProvided': 10000000,
 'winAgainstProfitLossPerShare': 0,
 'winForProfitLossPerShare': 0}

In [7]:
result.storage['liquidityForSharesLedger']

{('tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM', 0): 5000000}

In [8]:
result.storage['winAgainstProfitLossPerShareAtEntry']

{('tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM', 0): 0}

In [9]:
result = contract.bet({'bet': 'for', 'eventId': 0, 'minimalWinAmount': 1_000_000}).with_amount(1_000_000).interpret(
    sender=a, now=RUN_TIME + 6*ONE_HOUR, storage=result.storage)
result.storage['events'][0]

{'betsAgainstLiquidityPoolSum': 4166667,
 'betsCloseTime': 1620373402,
 'betsForLiquidityPoolSum': 6000000,
 'closedDynamics': 0,
 'closedOracleTime': 1530342452,
 'closedRate': 0,
 'createdTime': 1620287002,
 'currencyPair': 'XTZ-USD',
 'expirationFee': 200000,
 'firstProviderAgainstSharesSum': 5000000,
 'firstProviderForSharesSum': 5000000,
 'isBetsForWin': False,
 'isClosed': False,
 'isMeasurementStarted': False,
 'liquidityPercent': 10000,
 'liquidityPrecision': 1000000,
 'measureOracleStartTime': 1530342452,
 'measurePeriod': 43200,
 'measureStartFee': 200000,
 'oracleAddress': 'KT1SUP27JhX24Kvr11oUdWswk7FnCW78ZyUn',
 'ratioPrecision': 100000000,
 'rewardCallFee': 100000,
 'sharePrecision': 100000000,
 'startRate': 0,
 'targetDynamics': 1000000,
 'targetDynamicsPrecision': 1000000,
 'totalLiquidityAgainstSharesSum': 5000000,
 'totalLiquidityForSharesSum': 5000000,
 'totalLiquidityProvided': 10000000,
 'winAgainstProfitLossPerShare': 20000000,
 'winForProfitLossPerShare': -1649998

In [10]:
result = contract.provideLiquidity(provide_params).with_amount(10_000_000).interpret(
    sender=d, now=RUN_TIME + 6*ONE_HOUR, storage=result.storage)
result.storage['events'][0]

{'betsAgainstLiquidityPoolSum': 8265028,
 'betsCloseTime': 1620373402,
 'betsForLiquidityPoolSum': 11901639,
 'closedDynamics': 0,
 'closedOracleTime': 1530342452,
 'closedRate': 0,
 'createdTime': 1620287002,
 'currencyPair': 'XTZ-USD',
 'expirationFee': 200000,
 'firstProviderAgainstSharesSum': 5000000,
 'firstProviderForSharesSum': 5000000,
 'isBetsForWin': False,
 'isClosed': False,
 'isMeasurementStarted': False,
 'liquidityPercent': 10000,
 'liquidityPrecision': 1000000,
 'measureOracleStartTime': 1530342452,
 'measurePeriod': 43200,
 'measureStartFee': 200000,
 'oracleAddress': 'KT1SUP27JhX24Kvr11oUdWswk7FnCW78ZyUn',
 'ratioPrecision': 100000000,
 'rewardCallFee': 100000,
 'sharePrecision': 100000000,
 'startRate': 0,
 'targetDynamics': 1000000,
 'targetDynamicsPrecision': 1000000,
 'totalLiquidityAgainstSharesSum': 8073770,
 'totalLiquidityForSharesSum': 9426229,
 'totalLiquidityProvided': 20000000,
 'winAgainstProfitLossPerShare': 20000000,
 'winForProfitLossPerShare': -164999

In [11]:
result.storage['winAgainstProfitLossPerShareAtEntry']

{('tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM', 0): 0}

In [12]:
def get_params(storage, event_id, participant):
    key = (participant, event_id)
    win_for_expected = (
        storage['events'][event_id]['winForProfitLossPerShare']
        * storage['liquidityForSharesLedger'][key]
        / storage['events'][event_id]['sharePrecision'])

    win_against_expected = (
        storage['events'][event_id]['winAgainstProfitLossPerShare']
        * storage['liquidityAgainstSharesLedger'][key]
        / storage['events'][event_id]['sharePrecision'])

    # I calculate here without event:
    bets_for_winnig_sum = sum(storage['betsForWinningLedger'].values())
    bets_against_winning_sum = sum(storage['betsAgainstWinningLedger'].values())
    bets = sum(storage['depositedBets'].values())
    
    return {
        'win_for_expected': win_for_expected,
        'win_against_expected': win_against_expected,
        'bets_for_sum': bets_for_winnig_sum,
        'bets_against_sum': bets_against_winning_sum,
        'bets': bets
    }

In [13]:
get_params(result.storage, 0, d)

{'win_for_expected': -1555325.8997542,
 'win_against_expected': 1614754.0,
 'bets_for_sum': 1824999,
 'bets_against_sum': 0,
 'bets': 1000000}

In [14]:
result = contract.bet({'bet': 'against', 'eventId': 0, 'minimalWinAmount': 3_000_000}).with_amount(3_000_000).interpret(
    sender=b, now=RUN_TIME + 12*ONE_HOUR, storage=result.storage)
result.storage['events'][0]

{'betsAgainstLiquidityPoolSum': 11265028,
 'betsCloseTime': 1620373402,
 'betsForLiquidityPoolSum': 8732103,
 'closedDynamics': 0,
 'closedOracleTime': 1530342452,
 'closedRate': 0,
 'createdTime': 1620287002,
 'currencyPair': 'XTZ-USD',
 'expirationFee': 200000,
 'firstProviderAgainstSharesSum': 5000000,
 'firstProviderForSharesSum': 5000000,
 'isBetsForWin': False,
 'isClosed': False,
 'isMeasurementStarted': False,
 'liquidityPercent': 10000,
 'liquidityPrecision': 1000000,
 'measureOracleStartTime': 1530342452,
 'measurePeriod': 43200,
 'measureStartFee': 200000,
 'oracleAddress': 'KT1SUP27JhX24Kvr11oUdWswk7FnCW78ZyUn',
 'ratioPrecision': 100000000,
 'rewardCallFee': 100000,
 'sharePrecision': 100000000,
 'startRate': 0,
 'targetDynamics': 1000000,
 'targetDynamicsPrecision': 1000000,
 'totalLiquidityAgainstSharesSum': 8073770,
 'totalLiquidityForSharesSum': 9426229,
 'totalLiquidityProvided': 20000000,
 'winAgainstProfitLossPerShare': -13288391,
 'winForProfitLossPerShare': 153261

In [15]:
get_params(result.storage, 0, d)

{'win_for_expected': 1444674.03686732,
 'win_against_expected': -1072874.1260407,
 'bets_for_sum': 1824999,
 'bets_against_sum': 6137840,
 'bets': 4000000}

In [16]:
result.storage['winAgainstProfitLossPerShareAtEntry']

{('tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM', 0): 0}

In [17]:
result = contract.provideLiquidity(provide_params).with_amount(40_000_000).interpret(
    sender=c, now=RUN_TIME + 16*ONE_HOUR, storage=result.storage)

In [18]:
result.storage['liquidityForSharesLedger']

{('tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE', 0): 5822237,
 ('tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM', 0): 9426229}

In [19]:
result = contract.bet({'bet': 'for', 'eventId': 0, 'minimalWinAmount': 5_000_000}).with_amount(5_000_000).interpret(
    sender=a, now=RUN_TIME + 20*ONE_HOUR, storage=result.storage)
result.storage['events'][0]

{'betsAgainstLiquidityPoolSum': 29771930,
 'betsCloseTime': 1620372978,
 'betsForLiquidityPoolSum': 29518285,
 'closedDynamics': 0,
 'closedOracleTime': 1530342452,
 'closedRate': 0,
 'createdTime': 1620286578,
 'currencyPair': 'XTZ-USD',
 'expirationFee': 200000,
 'firstProviderAgainstSharesSum': 5000000,
 'firstProviderForSharesSum': 5000000,
 'isBetsForWin': False,
 'isClosed': False,
 'isMeasurementStarted': False,
 'liquidityPercent': 10000,
 'liquidityPrecision': 1000000,
 'measureOracleStartTime': 1530342452,
 'measurePeriod': 43200,
 'measureStartFee': 200000,
 'oracleAddress': 'KT1SUP27JhX24Kvr11oUdWswk7FnCW78ZyUn',
 'ratioPrecision': 100000000,
 'rewardCallFee': 100000,
 'sharePrecision': 100000000,
 'startRate': 0,
 'targetDynamics': 1000000,
 'targetDynamicsPrecision': 1000000,
 'totalLiquidityAgainstSharesSum': 16667464,
 'totalLiquidityForSharesSum': 14165869,
 'totalLiquidityProvided': 60000000,
 'winAgainstProfitLossPerShare': 10546804,
 'winForProfitLossPerShare': -120

In [20]:
get_params(result.storage, 0, d)

{'win_for_expected': -1055443.375,
 'win_against_expected': 922845.35,
 'bets_for_sum': 12910657,
 'bets_against_sum': 5702029,
 'bets': 9000000}

In [21]:
result = contract.bet({'bet': 'against', 'eventId': 0, 'minimalWinAmount': 5_000_000}).with_amount(5_000_000).interpret(
    sender=c, now=RUN_TIME + 22*ONE_HOUR, storage=result.storage)
result.storage['events'][0]

{'betsAgainstLiquidityPoolSum': 34771930,
 'betsCloseTime': 1620372978,
 'betsForLiquidityPoolSum': 25273729,
 'closedDynamics': 0,
 'closedOracleTime': 1530342452,
 'closedRate': 0,
 'createdTime': 1620286578,
 'currencyPair': 'XTZ-USD',
 'expirationFee': 200000,
 'firstProviderAgainstSharesSum': 5000000,
 'firstProviderForSharesSum': 5000000,
 'isBetsForWin': False,
 'isClosed': False,
 'isMeasurementStarted': False,
 'liquidityPercent': 10000,
 'liquidityPrecision': 1000000,
 'measureOracleStartTime': 1530342452,
 'measurePeriod': 43200,
 'measureStartFee': 200000,
 'oracleAddress': 'KT1SUP27JhX24Kvr11oUdWswk7FnCW78ZyUn',
 'ratioPrecision': 100000000,
 'rewardCallFee': 100000,
 'sharePrecision': 100000000,
 'startRate': 0,
 'targetDynamics': 1000000,
 'targetDynamicsPrecision': 1000000,
 'totalLiquidityAgainstSharesSum': 16667464,
 'totalLiquidityForSharesSum': 14165869,
 'totalLiquidityProvided': 60000000,
 'winAgainstProfitLossPerShare': -19116819,
 'winForProfitLossPerShare': 232

In [22]:
result = contract.startMeasurement(0).interpret(
    sender=a, storage=result.storage, now=RUN_TIME + 26*ONE_HOUR)
result.storage['events'][0]

{'betsAgainstLiquidityPoolSum': 34771930,
 'betsCloseTime': 1620372978,
 'betsForLiquidityPoolSum': 25273729,
 'closedDynamics': 0,
 'closedOracleTime': 1530342452,
 'closedRate': 0,
 'createdTime': 1620286578,
 'currencyPair': 'XTZ-USD',
 'expirationFee': 200000,
 'firstProviderAgainstSharesSum': 5000000,
 'firstProviderForSharesSum': 5000000,
 'isBetsForWin': False,
 'isClosed': False,
 'isMeasurementStarted': False,
 'liquidityPercent': 10000,
 'liquidityPrecision': 1000000,
 'measureOracleStartTime': 1530342452,
 'measurePeriod': 43200,
 'measureStartFee': 200000,
 'oracleAddress': 'KT1SUP27JhX24Kvr11oUdWswk7FnCW78ZyUn',
 'ratioPrecision': 100000000,
 'rewardCallFee': 100000,
 'sharePrecision': 100000000,
 'startRate': 0,
 'targetDynamics': 1000000,
 'targetDynamicsPrecision': 1000000,
 'totalLiquidityAgainstSharesSum': 16667464,
 'totalLiquidityForSharesSum': 14165869,
 'totalLiquidityProvided': 60000000,
 'winAgainstProfitLossPerShare': -19116819,
 'winForProfitLossPerShare': 232

In [23]:
callback = {
    'currencyPair': 'XTZ-USD',
    'lastUpdate': RUN_TIME + 26*ONE_HOUR - 1*ONE_HOUR,
    'rate': 6_000_000
}

result = contract.startMeasurementCallback(callback).interpret(
    sender=event['oracleAddress'], now=RUN_TIME + 26*ONE_HOUR, storage=result.storage)

In [24]:
result = contract.close(0).interpret(
    sender=a, storage=result.storage, now=RUN_TIME + 38*ONE_HOUR)

In [25]:
callback = {
    'currencyPair': 'XTZ-USD',
    'lastUpdate': RUN_TIME + 38*ONE_HOUR - 1*ONE_HOUR,
    'rate': 7_500_000
}

result = contract.closeCallback(callback).interpret(
    sender=event['oracleAddress'], now=RUN_TIME + 38*ONE_HOUR, storage=result.storage)

In [26]:
def remove_none_values(storage):
    def clean_dict(dct):
        return {key: value for key, value in dct.items() if value}

    return {
        key: clean_dict(value) if type(value) is dict else value
        for key, value in storage.items()
    }

In [28]:
result.storage['betsForWinningLedger']

{('tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ', 0): 12910657}

In [29]:
result.storage['betsAgainstWinningLedger']

{('tz1MdaJfWzP5pPx3gwPxfdLZTHW6js9havos', 0): 5702029,
 ('tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE', 0): 9202110}

In [30]:
c

'tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE'

In [31]:
result = contract.withdraw(0).interpret(
    sender=c, now=RUN_TIME + 40*ONE_HOUR, storage=result.storage)
result.storage = remove_none_values(result.storage)

result.operations

[{'kind': 'transaction',
  'source': 'KT1BEqzn5Wx8uJrZNvuS9DVHmLvG9td3fDLi',
  'destination': 'tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE',
  'amount': '76790705',
  'parameters': {'entrypoint': 'default', 'value': {'prim': 'Unit'}}}]

In [33]:
result.storage['betsForWinningLedger']

{('tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ', 0): 12910657}

In [34]:
result.storage['betsAgainstWinningLedger']

{('tz1MdaJfWzP5pPx3gwPxfdLZTHW6js9havos', 0): 5702029}

In [36]:
result = contract.withdraw(0).interpret(
    sender=a, now=RUN_TIME + 40*ONE_HOUR, storage=result.storage)
result.storage = remove_none_values(result.storage)

In [38]:
result.operations

[{'kind': 'transaction',
  'source': 'KT1BEqzn5Wx8uJrZNvuS9DVHmLvG9td3fDLi',
  'destination': 'tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ',
  'amount': '12910657',
  'parameters': {'entrypoint': 'default', 'value': {'prim': 'Unit'}}}]

### Nohting to withdraw because b is not withdrawn:

In [39]:
result = contract.withdraw(0).interpret(
    sender=d, now=RUN_TIME + 40*ONE_HOUR, storage=result.storage)
result.storage = remove_none_values(result.storage)

In [40]:
result = contract.withdraw(0).interpret(
    sender=b, now=RUN_TIME + 40*ONE_HOUR, storage=result.storage, balance=64_000_000)
result.storage = remove_none_values(result.storage)

In [41]:
result.operations

[{'kind': 'transaction',
  'source': 'KT1BEqzn5Wx8uJrZNvuS9DVHmLvG9td3fDLi',
  'destination': 'tz1MdaJfWzP5pPx3gwPxfdLZTHW6js9havos',
  'amount': '0',
  'parameters': {'entrypoint': 'default', 'value': {'prim': 'Unit'}}}]

In [42]:
result = contract.withdraw(0).interpret(
    sender=d, now=RUN_TIME + 40*ONE_HOUR, storage=result.storage, balance=64_000_000)
result.storage = remove_none_values(result.storage)
result.operations

[{'kind': 'transaction',
  'source': 'KT1BEqzn5Wx8uJrZNvuS9DVHmLvG9td3fDLi',
  'destination': 'tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM',
  'amount': '0',
  'parameters': {'entrypoint': 'default', 'value': {'prim': 'Unit'}}}]

In [44]:
result.storage

{'betsAgainstWinningLedger': {},
 'betsForWinningLedger': {},
 'closeCallEventId': None,
 'depositedBets': {('tz1MdaJfWzP5pPx3gwPxfdLZTHW6js9havos', 0): 3000000,
  ('tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE', 0): 5000000,
  ('tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ', 0): 6000000},
 'events': {0: {'betsAgainstLiquidityPoolSum': 34771930,
   'betsCloseTime': 1620372978,
   'betsForLiquidityPoolSum': 25273729,
   'closedDynamics': 1250000,
   'closedOracleTime': 1620419778,
   'closedRate': 7500000,
   'createdTime': 1620286578,
   'currencyPair': 'XTZ-USD',
   'expirationFee': 200000,
   'firstProviderAgainstSharesSum': 5000000,
   'firstProviderForSharesSum': 5000000,
   'isBetsForWin': True,
   'isClosed': True,
   'isMeasurementStarted': True,
   'liquidityPercent': 10000,
   'liquidityPrecision': 1000000,
   'measureOracleStartTime': 1620376578,
   'measurePeriod': 43200,
   'measureStartFee': 200000,
   'oracleAddress': 'KT1SUP27JhX24Kvr11oUdWswk7FnCW78ZyUn',
   'ratioPrecision': 100000000

### TODO: тесты
- тест что при ставке рейт оказывается хуже ожидаемого
- тест что при добавлении ликвидности рейт хуже ожидаемого
- тест что попытка добавить ликвидность после окончания времени
- тест что один участник одновременно и провайдер и игрок
- тест что провайдер выводит средства за игроков
- тест сценария когда провайдеры терпят убытки
- что провайдер не может вывести средства свои пока есть участники
